# Introdução às Redes Neurais Convolucionais em Grafos para Séries Temporais

As **Redes Neurais Convolucionais em Grafos** (Graph Convolutional Networks, ou GCNs) têm se mostrado uma ferramenta poderosa para capturar padrões complexos em dados estruturados em forma de grafos, como redes de transporte, redes sociais e interações moleculares. No entanto, muitos problemas no mundo real também envolvem uma dimensão temporal, onde as relações entre as entidades mudam com o tempo. Isso exige a criação de modelos capazes de capturar **dependências espaciais** e **temporais** simultaneamente.

A combinação dessas duas dimensões resulta em uma nova classe de problemas conhecidos como **Redes Neurais Espaciais-Temporais em Grafos** (Spatio-Temporal Graph Neural Networks, ou ST-GNNs). Esses modelos são aplicados em uma variedade de tarefas, como previsão de tráfego, análise de redes de sensores e modelagem de interações dinâmicas em redes sociais.

Nesta apresentação, exploraremos como a GCN pode ser expandida para lidar com séries temporais, capturando tanto a estrutura do grafo quanto as mudanças ao longo do tempo. Vamos focar em exemplos práticos, como a **previsão de tráfego**, onde o tráfego entre diferentes segmentos de estrada é modelado como um grafo cujas características variam ao longo do tempo.

## Definição

Dado um grafo dinâmico $G(t) = (V, E, X_V(t), X_E(t))$, onde:

- $V$ é o conjunto de nós (vértices),
- $E$ é o conjunto de arestas (conexões entre os nós),
- $X_V(t) \in \mathbb{R}^{|V| \times d_V}$ representa as características dos nós (vértices) no tempo $t$, sendo $d_V$ a dimensionalidade das características dos nós,
- $X_E(t) \in \mathbb{R}^{|E| \times d_E}$ representa as características das arestas no tempo $t$, sendo $d_E$ a dimensionalidade das características das arestas.

![example](st_gnn.png)

# Motivação: Previsão de Trafégo

Neste exemplo, mostramos como prever as condições de tráfego utilizando redes neurais baseadas em grafos **(GNNs)** combinadas com **LSTM**. O foco é prever os valores futuros da velocidade do tráfego com base em um histórico de dados de uma coleção de segmentos de estrada.

Uma solução tradicional para este tipo de problema é tratar a velocidade de tráfego de cada segmento de estrada como uma série temporal individual, prevendo os valores futuros com base nos dados passados do próprio segmento. Entretanto, essa abordagem não considera as influências que os segmentos vizinhos exercem entre si.

Para capturar as interações complexas entre os diferentes segmentos de estrada, podemos modelar a rede de tráfego como um grafo, onde a velocidade do tráfego é representada como um sinal associado a esse grafo. Nesta abordagem, implementamos uma arquitetura de rede neural capaz de processar séries temporais sobre um grafo. Primeiramente, mostramos como preparar os dados e criar um `tf.data.Dataset` para previsão com grafos. Em seguida, desenvolvemos um modelo que combina camadas de convolução em grafos com LSTM para realizar a previsão.